T5 GENERATION & EVALUATION - FINETUNED MODEL - PROCESSED TEST SET

Load in test data (samples 10 and 500).
Build my own model and load in weights from 3rd epoch of finetuning training.
Feed the finetuned t5 model a pre-processed test set and generate text. Use the selected hyperparameters for model.generate().
Do this is batches and then concat. datasets to get rouge results.

In [1]:
# mount to colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -q rouge_score
!pip install -q evaluate
import numpy as np
import evaluate
import rouge_score
import pandas as pd

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.1/519.1 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.0 MB/s eta 0:00:00


In [3]:
rouge = evaluate.load('rouge')

In [4]:
!pip install transformers
from transformers import AutoTokenizer  , TFAutoModel
from transformers import T5Tokenizer, TFT5ForConditionalGeneration
from pprint import pprint

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 64.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 99.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.0 MB/s eta 0:00:00


In [5]:
# get regular test data
test_file = "drive/MyDrive/266project/billsum_test.csv"
test_data = pd.read_csv(test_file)

In [6]:
#take a sample of 10 for testing my code, then a sample of 500 for the actual experiment
test_data_10 = test_data.sample(n=10, random_state=0)
test_data_500 = test_data.sample(n=500, random_state=0)
test_data_10 = test_data_10.reset_index(drop=True)
test_data_500 = test_data_500.reset_index(drop=True)

In [7]:
# load out of box model
t5_model_name = 't5-base'
orig_t5_model = TFT5ForConditionalGeneration.from_pretrained(t5_model_name)
tokenizer = AutoTokenizer.from_pretrained(t5_model_name)

All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [8]:
# get my own model
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

def build_t5_training_wrapper_model(my_t5, max_length):
    input_ids = layers.Input(shape=(max_length), dtype=tf.int32, name='input_ids')
    attention_mask = layers.Input(shape=(max_length), dtype=tf.int32, name='attention_mask')
    decoder_input_ids = layers.Input(shape=(max_length), dtype=tf.int32, name='labels')

    t5_logits = my_t5(input_ids, attention_mask=attention_mask, decoder_input_ids=decoder_input_ids)[0]

    model = tf.keras.models.Model(inputs=[input_ids, attention_mask, decoder_input_ids],
                                  outputs=[t5_logits])
    model.compile(optimizer=tf.keras.optimizers.Adam(),
                  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True))

    return model

In [9]:
max_length = 168
checkpoint_dir = 'drive/MyDrive/266project/model_checkpoints/'
latest_model = 't5_billsum_weights.03.hdf5'
finetuned_model = orig_t5_model
model_wrapper = build_t5_training_wrapper_model(finetuned_model, max_length)
model_wrapper.load_weights(checkpoint_dir + latest_model)

In [10]:
# function for generation with better HPS
def generate_candidates_HPs(orig, model):
  """Test data is a pandas df, "orig" is test_df.orig.
  Adjusted hyperparameters are used.
  """
  input = tokenizer(orig, return_tensors = 'tf',padding = True).input_ids
  output = model.generate(input,
                          max_new_tokens = 150,
                          min_length = 50,
                          num_beams = 1,
                          no_repeat_ngram_size = 2
                          )
  decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)
  return decoded_output

In [11]:
# Pre_processing function to improve test set input based on structure of the docs

keywords = ["SECTION", "\nSECTION" ,"SEC", "\nSEC" "FINDINGS"]
n = 50 # is the number of words after a keyword that we will take
def process(bill, n):
  new_bill = ''
  bill_words = bill.split()
  for idx in range(len(bill_words)):
    for i in range(len(keywords)):
      if keywords[i] in bill_words[idx]:
        important_text = " ".join(bill_words[(idx + 2): (idx + 2 + n + 1)])
        new_bill = new_bill + important_text
  return new_bill


# do this function for every row in test_data and then output a df with a column called "procesed_bill"


In [12]:
first_example = test_data_10.head(1)

In [13]:
# run code on just one example first
first_example['custom_processed'] = first_example.apply(lambda row: process(row.orig,n), axis = 1)

<ipython-input-13-3f3676d46e51>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first_example['custom_processed'] = first_example.apply(lambda row: process(row.orig,n), axis = 1)


In [14]:
PROMPT = 'summarize: '
first_example['t5_finetuned_candidate_on_processed'] = first_example.apply(lambda x: generate_candidates_HPs(PROMPT + x.custom_processed, finetuned_model), axis = 1)

<ipython-input-14-ba6422c13299>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first_example['t5_finetuned_candidate_on_processed'] = first_example.apply(lambda x: generate_candidates_HPs(PROMPT + x.custom_processed, finetuned_model), axis = 1)


In [ ]:
pd.set_option('display.max_colwidth', None)
first_example

In [16]:
# create dataframes with the chunks that we will use to generate candidates
df_1_p = test_data_500[:50]
df_2_p = test_data_500[50:100]
df_3_p = test_data_500[100:150]
df_4_p = test_data_500[150:200]
df_5_p = test_data_500[200:250]
df_6_p = test_data_500[250:300]
df_7_p = test_data_500[300:350]
df_8_p = test_data_500[350:400]
df_9_p = test_data_500[400:450]
df_10_p = test_data_500[450:500]

In [ ]:
PROMPT = 'summarize: '
df_1_p['custom_processed'] = df_1_p.apply(lambda row: process(row.orig,n), axis = 1)
df_1_p['t5_finetuned_candidate_on_processed'] = df_1_p.apply(lambda x: generate_candidates_HPs(PROMPT + x.custom_processed, finetuned_model), axis = 1)
df_1_p.to_csv('drive/MyDrive/266project/dfs_finetuned_unprocessed/df_1_p.csv')

<ipython-input-19-8c19ccc0595f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1_p['custom_processed'] = df_1_p.apply(lambda row: process(row.orig,n), axis = 1)
Token indices sequence length is longer than the specified maximum sequence length for this model (647 > 512). Running this sequence through the model will result in indexing errors


In [20]:
PROMPT = 'summarize: '
df_2_p['custom_processed'] = df_2_p.apply(lambda row: process(row.orig,n), axis = 1)
df_2_p['t5_finetuned_candidate_on_processed'] = df_2_p.apply(lambda x: generate_candidates_HPs(PROMPT + x.custom_processed, finetuned_model), axis = 1)
df_2_p.to_csv('drive/MyDrive/266project/dfs_finetuned_unprocessed/df_2_p.csv')

<ipython-input-20-eda4386651da>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2_p['custom_processed'] = df_2_p.apply(lambda row: process(row.orig,n), axis = 1)
<ipython-input-20-eda4386651da>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2_p['t5_finetuned_candidate_on_processed'] = df_2_p.apply(lambda x: generate_candidates_HPs(PROMPT + x.custom_processed, finetuned_model), axis = 1)


In [21]:
PROMPT = 'summarize: '
df_3_p['custom_processed'] = df_3_p.apply(lambda row: process(row.orig,n), axis = 1)
df_3_p['t5_finetuned_candidate_on_processed'] = df_3_p.apply(lambda x: generate_candidates_HPs(PROMPT + x.custom_processed, finetuned_model), axis = 1)
df_3_p.to_csv('drive/MyDrive/266project/dfs_finetuned_unprocessed/df_3_p.csv')

<ipython-input-21-eff2291c3141>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_3_p['custom_processed'] = df_3_p.apply(lambda row: process(row.orig,n), axis = 1)
<ipython-input-21-eff2291c3141>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_3_p['t5_finetuned_candidate_on_processed'] = df_3_p.apply(lambda x: generate_candidates_HPs(PROMPT + x.custom_processed, finetuned_model), axis = 1)


In [22]:
PROMPT = 'summarize: '
df_4_p['custom_processed'] = df_4_p.apply(lambda row: process(row.orig,n), axis = 1)
df_4_p['t5_finetuned_candidate_on_processed'] = df_4_p.apply(lambda x: generate_candidates_HPs(PROMPT + x.custom_processed, finetuned_model), axis = 1)
df_4_p.to_csv('drive/MyDrive/266project/dfs_finetuned_unprocessed/df_4_p.csv')

<ipython-input-22-36728b63e8e2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_4_p['custom_processed'] = df_4_p.apply(lambda row: process(row.orig,n), axis = 1)
<ipython-input-22-36728b63e8e2>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_4_p['t5_finetuned_candidate_on_processed'] = df_4_p.apply(lambda x: generate_candidates_HPs(PROMPT + x.custom_processed, finetuned_model), axis = 1)


In [23]:
PROMPT = 'summarize: '
df_5_p['custom_processed'] = df_5_p.apply(lambda row: process(row.orig,n), axis = 1)
df_5_p['t5_finetuned_candidate_on_processed'] = df_5_p.apply(lambda x: generate_candidates_HPs(PROMPT + x.custom_processed, finetuned_model), axis = 1)
df_5_p.to_csv('drive/MyDrive/266project/dfs_finetuned_unprocessed/df_5_p.csv')

<ipython-input-23-dbe8c402534b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_5_p['custom_processed'] = df_5_p.apply(lambda row: process(row.orig,n), axis = 1)
<ipython-input-23-dbe8c402534b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_5_p['t5_finetuned_candidate_on_processed'] = df_5_p.apply(lambda x: generate_candidates_HPs(PROMPT + x.custom_processed, finetuned_model), axis = 1)


In [24]:
PROMPT = 'summarize: '
df_6_p['custom_processed'] = df_6_p.apply(lambda row: process(row.orig,n), axis = 1)
df_6_p['t5_finetuned_candidate_on_processed'] = df_6_p.apply(lambda x: generate_candidates_HPs(PROMPT + x.custom_processed, finetuned_model), axis = 1)
df_6_p.to_csv('drive/MyDrive/266project/dfs_finetuned_unprocessed/df_6_p.csv')

<ipython-input-24-f88ab8cd28e3>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_6_p['custom_processed'] = df_6_p.apply(lambda row: process(row.orig,n), axis = 1)
<ipython-input-24-f88ab8cd28e3>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_6_p['t5_finetuned_candidate_on_processed'] = df_6_p.apply(lambda x: generate_candidates_HPs(PROMPT + x.custom_processed, finetuned_model), axis = 1)


In [ ]:
PROMPT = 'summarize: '
df_7_p['custom_processed'] = df_7_p.apply(lambda row: process(row.orig,n), axis = 1)
df_7_p['t5_finetuned_candidate_on_processed'] = df_7_p.apply(lambda x: generate_candidates_HPs(PROMPT + x.custom_processed, finetuned_model), axis = 1)
df_7_p.to_csv('drive/MyDrive/266project/dfs_finetuned_unprocessed/df_7_p.csv')

<ipython-input-25-122087d12747>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_7_p['custom_processed'] = df_7_p.apply(lambda row: process(row.orig,n), axis = 1)


In [26]:
PROMPT = 'summarize: '
df_8_p['custom_processed'] = df_8_p.apply(lambda row: process(row.orig,n), axis = 1)
df_8_p['t5_finetuned_candidate_on_processed'] = df_8_p.apply(lambda x: generate_candidates_HPs(PROMPT + x.custom_processed, finetuned_model), axis = 1)
df_8_p.to_csv('drive/MyDrive/266project/dfs_finetuned_unprocessed/df_8_p.csv')

<ipython-input-26-ea0a2a359976>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_8_p['custom_processed'] = df_8_p.apply(lambda row: process(row.orig,n), axis = 1)
<ipython-input-26-ea0a2a359976>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_8_p['t5_finetuned_candidate_on_processed'] = df_8_p.apply(lambda x: generate_candidates_HPs(PROMPT + x.custom_processed, finetuned_model), axis = 1)


In [27]:
PROMPT = 'summarize: '
df_9_p['custom_processed'] = df_9_p.apply(lambda row: process(row.orig,n), axis = 1)
df_9_p['t5_finetuned_candidate_on_processed'] = df_9_p.apply(lambda x: generate_candidates_HPs(PROMPT + x.custom_processed, finetuned_model), axis = 1)
df_9_p.to_csv('drive/MyDrive/266project/dfs_finetuned_unprocessed/df_9_p.csv')

<ipython-input-27-09919b1b87b8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_9_p['custom_processed'] = df_9_p.apply(lambda row: process(row.orig,n), axis = 1)
<ipython-input-27-09919b1b87b8>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_9_p['t5_finetuned_candidate_on_processed'] = df_9_p.apply(lambda x: generate_candidates_HPs(PROMPT + x.custom_processed, finetuned_model), axis = 1)


In [28]:
PROMPT = 'summarize: '
df_10_p['custom_processed'] = df_10_p.apply(lambda row: process(row.orig,n), axis = 1)
df_10_p['t5_finetuned_candidate_on_processed'] = df_10_p.apply(lambda x: generate_candidates_HPs(PROMPT + x.custom_processed, finetuned_model), axis = 1)
df_10_p.to_csv('drive/MyDrive/266project/dfs_finetuned_unprocessed/df_10_p.csv')

<ipython-input-28-ba5bb4d88796>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_10_p['custom_processed'] = df_10_p.apply(lambda row: process(row.orig,n), axis = 1)
<ipython-input-28-ba5bb4d88796>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_10_p['t5_finetuned_candidate_on_processed'] = df_10_p.apply(lambda x: generate_candidates_HPs(PROMPT + x.custom_processed, finetuned_model), axis = 1)


In [29]:
#load all the files in
df_1_p_done = pd.read_csv('drive/MyDrive/266project/dfs_finetuned_unprocessed/df_1_p.csv')
df_2_p_done = pd.read_csv('drive/MyDrive/266project/dfs_finetuned_unprocessed/df_2_p.csv')
df_3_p_done = pd.read_csv('drive/MyDrive/266project/dfs_finetuned_unprocessed/df_3_p.csv')
df_4_p_done = pd.read_csv('drive/MyDrive/266project/dfs_finetuned_unprocessed/df_4_p.csv')
df_5_p_done = pd.read_csv('drive/MyDrive/266project/dfs_finetuned_unprocessed/df_5_p.csv')
df_6_p_done = pd.read_csv('drive/MyDrive/266project/dfs_finetuned_unprocessed/df_6_p.csv')
df_7_p_done = pd.read_csv('drive/MyDrive/266project/dfs_finetuned_unprocessed/df_7_p.csv')
df_8_p_done = pd.read_csv('drive/MyDrive/266project/dfs_finetuned_unprocessed/df_8_p.csv')
df_9_p_done = pd.read_csv('drive/MyDrive/266project/dfs_finetuned_unprocessed/df_9_p.csv')
df_10_p_done = pd.read_csv('drive/MyDrive/266project/dfs_finetuned_unprocessed/df_10_p.csv')

In [30]:
all_dfs = pd.concat([df_1_p_done, df_2_p_done, df_3_p_done, df_4_p_done, df_5_p_done, df_6_p_done, df_7_p_done, df_8_p_done, df_9_p_done, df_10_p_done], ignore_index=True)

In [31]:
len(all_dfs)

500

In [32]:
t5_rouge_finetuned_processed = rouge.compute(predictions = all_dfs['t5_finetuned_candidate_on_processed'],
                       references= all_dfs['target'])
t5_rouge_finetuned_processed

{'rouge1': 0.22615078727784155,
 'rouge2': 0.09136248916113524,
 'rougeL': 0.174652755223017,
 'rougeLsum': 0.18657879294680746}